In [1]:
import gc
import numpy as np 
import pandas as pd
import optuna
from sklearn.metrics import log_loss
import warnings

warnings.filterwarnings('ignore')



In [2]:
%%time
h5File = "temp_data/h5/selectkbest_data_with_last.h5";
key_train = 'bestkselect_01/train';
key_valid = 'bestkselect_01/valid';
key_test = 'bestkselect_01/test';

train = pd.read_hdf(h5File, key_train);
valid = pd.read_hdf(h5File, key_valid);
test = pd.read_hdf(h5File, key_test);


CPU times: user 11.3 s, sys: 16.1 s, total: 27.4 s
Wall time: 32.7 s


In [3]:
train

,P_2_min,P_2_max,P_2_avg,D_39_max,D_39_avg,B_1_min,B_1_max,B_1_avg,B_2_min,B_2_max,...,D_132_last,D_132_ldiff,D_133_last,D_136_last,D_139_last,D_140_last,D_141_last,D_143_last,D_145_last,S_2_min_m
customer_ID,,,,,,,,,,,,,,,,,,,,,
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0.868580,0.960384,0.933824,0.091505,0.010704,0.001930,0.021655,0.012007,1.000242,1.009672,...,0.0,0.0,0.006210,0.000000,0.007186,0.004234,0.005086,0.005810,0.008533,3
00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5,0.861109,0.929122,0.899820,0.567403,0.215205,0.006711,0.109644,0.025654,0.819772,1.008534,...,0.0,0.0,0.002996,0.000000,0.002980,0.007479,0.007870,0.003284,0.008514,3
00001b22f846c82c51f6e3958ccd81970162bae8b007e80662ef27519fcc18c1,0.797670,0.904482,0.878454,0.009704,0.004181,0.001472,0.009997,0.004386,0.810796,0.819987,...,0.0,0.0,0.009881,0.000000,0.007383,0.006623,0.000964,0.002202,0.003444,3
000041bdba6ecadd89a52d11886e8eaaec9325906c9723355abb5ca523658edc,0.567442,0.623392,0.598969,0.268476,0.048862,0.005910,0.279991,0.059876,0.812053,1.009999,...,0.0,0.0,0.001789,0.000000,0.002704,0.006184,0.001899,0.008183,0.002983,3
00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8ad51ca8b8c4a24cefed,0.805045,0.940382,0.891679,0.008680,0.004644,0.000776,0.009806,0.005941,0.810670,0.819947,...,0.0,0.0,0.005045,0.000000,0.002974,0.004162,0.005764,0.008154,0.000905,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
e0dc8401f13e88ae5fadf2a6cd22ec8f2162a75964a00492ad406dfd437e716f,0.036622,0.428700,0.305832,0.765157,0.145896,0.053321,0.204184,0.145427,0.051535,0.814995,...,0.0,0.0,0.004369,0.000000,0.000320,0.004045,0.002236,0.004224,0.002413,7
e0dcb868e734d5abe60f6a0d9107989aa0cef45c7da5f18642dee0b455c6c311,0.164876,0.384758,0.312491,0.766661,0.255978,0.451088,1.322943,0.693505,0.008950,0.038503,...,0.0,0.0,0.000537,0.000000,0.005447,0.002212,0.008570,0.009164,0.007635,3
e0dcc67c6a5854ab27ac3b799141c2a5c9831471b93a96f1ee62c3e556133956,0.520425,0.677117,0.582655,1.036184,0.308401,0.111283,0.498702,0.325557,0.050609,0.433234,...,0.0,0.0,0.005766,0.000000,0.000154,0.007504,0.007130,0.006720,0.003733,3


In [4]:
valid

customer_ID
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a    0.0
00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5    0.0
00001b22f846c82c51f6e3958ccd81970162bae8b007e80662ef27519fcc18c1    0.0
000041bdba6ecadd89a52d11886e8eaaec9325906c9723355abb5ca523658edc    0.0
00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8ad51ca8b8c4a24cefed    0.0
                                                                   ... 
e0dc8401f13e88ae5fadf2a6cd22ec8f2162a75964a00492ad406dfd437e716f    1.0
e0dcb868e734d5abe60f6a0d9107989aa0cef45c7da5f18642dee0b455c6c311    1.0
e0dcc67c6a5854ab27ac3b799141c2a5c9831471b93a96f1ee62c3e556133956    0.0
e0dd131940680fb03f52d808f3c1308abf4b057188fc0540614d2a543ab8a1ea    0.0
e0dd17ec1fae28e858808935b974ce142d53238d657018a1a2f129335e62d388    1.0
Name: target, Length: 458913, dtype: float64

In [5]:
test

,P_2_min,P_2_max,P_2_avg,D_39_max,D_39_avg,B_1_min,B_1_max,B_1_avg,B_2_min,B_2_max,...,D_132_last,D_132_ldiff,D_133_last,D_136_last,D_139_last,D_140_last,D_141_last,D_143_last,D_145_last,S_2_min_m
customer_ID,,,,,,,,,,,,,,,,,,,,,
00000469ba478561f23a92a868bd366de6f6527a684c9a2e78fb826dcac3b9b7,0.568930,0.631315,0.601387,0.238794,0.069758,0.007959,0.021672,0.013925,0.810456,1.009347,...,0.000000,0.000000,0.006273,0.0,0.005912,0.001250,0.006543,0.009160,0.003219,2
00001bf2e77ff879fab36aa4fac689b9ba411dae63ae397d4263dafa1daedef5,0.794469,0.913501,0.862166,0.505795,0.154252,0.014187,0.276014,0.052342,1.000782,1.009245,...,0.000000,0.000000,0.002767,0.0,0.004344,0.000866,0.009120,0.002196,0.007780,4
0000210045da4f81e5f122c6bde5c2a617d03eef67f82c5e400fc98e7bd43ce8,0.673112,0.835114,0.748955,0.679547,0.181814,0.001483,0.039697,0.012762,0.810072,1.009582,...,0.000000,0.000000,0.002045,0.0,1.001246,0.008894,0.896224,1.009391,0.092041,10
00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976cf6e56734528702d694,0.428457,0.514222,0.474728,0.683640,0.470512,0.149511,0.309129,0.284608,0.032696,0.205678,...,0.000000,0.000000,0.009377,0.0,1.008246,0.003753,0.919774,1.008100,0.183020,4
00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9a4693dd914fca22557,0.254478,0.425764,0.324100,0.768016,0.353050,0.357828,0.563603,0.456779,0.020167,0.106739,...,0.000000,0.000000,0.007940,0.0,0.006623,0.001140,0.009527,0.009408,0.000525,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c5d60460dba6dedc41e,0.597667,0.646915,0.631613,0.009711,0.003932,0.001874,0.011309,0.007622,0.810936,0.819394,...,0.000000,0.000000,0.009844,0.0,0.003015,0.006851,0.009307,0.009393,0.005294,4
ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3a4f0ca3de613b0b2ad,0.471303,0.634783,0.547049,0.417259,0.056910,0.015243,0.084163,0.049328,0.082507,1.009882,...,0.000000,0.000000,0.001985,0.0,0.007684,0.003375,0.009780,0.008109,0.009981,10
ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475cb095d2443a68030f1,0.195671,0.316648,0.239070,1.119398,0.232943,0.013811,0.051597,0.031747,0.171140,1.009973,...,0.145253,0.053708,0.274885,0.0,0.002304,0.001640,0.002371,0.008154,0.002120,4


In [6]:
X_train = train
y_train = valid
X_test = test

In [7]:
# X_train = train[:1000]
# y_train = valid[:1000]
# X_test = test[:1000]

In [14]:
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=0, stratify=y_train)
print(X_tr.head())
print(X_val.head())
print(y_tr.head())
print(y_val.head())

                                                     P_2_min   P_2_max  \
customer_ID                                                              
002972bc21889b65af21667afbeb986fa4fa09746930884...  0.718228  0.880460   
007bab3853dfbc1ed707891b73e5f1d2e176ee8511df473...  0.314582  0.585926   
00482d92a56c79e950955b090f389983df02b9e51617920...  0.251935  0.538769   
003d73b980a0824c3aace3feb54d97c0b30aa926dee9ef2...  0.266275  0.348925   
002b930ad6a7e6551482b48a4dbf0f43003e80877f72244...  0.469002  0.547550   

                                                     P_2_avg  D_39_max  \
customer_ID                                                              
002972bc21889b65af21667afbeb986fa4fa09746930884...  0.811594  0.742716   
007bab3853dfbc1ed707891b73e5f1d2e176ee8511df473...  0.468063  0.980343   
00482d92a56c79e950955b090f389983df02b9e51617920...  0.352982  1.037869   
003d73b980a0824c3aace3feb54d97c0b30aa926dee9ef2...  0.321002  0.325461   
002b930ad6a7e6551482b48a4dbf0f43003e8

In [15]:
from kerastuner.tuners import RandomSearch
import kerastuner as kt
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf

def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
        if hp.Choice('batchnorm_and_dropout', ['batch', 'dropout', 'both']) == 'batch':
            model.add(layers.BatchNormalization())
        elif hp.Choice('batchnorm_and_dropout', ['batch', 'dropout', 'both']) == 'dropout':
            model.add(layers.Dropout(0.2))
        else:
            model.add(layers.BatchNormalization())
            model.add(layers.Dropout(0.2))
    model.add(layers.Dense(1, activation="sigmoid"))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss=keras.losses.BinaryCrossentropy(from_logits=True),
        metrics=[keras.metrics.BinaryAccuracy(name='acc')])
    return model


In [19]:
tuner = RandomSearch(
    build_model,
    objective='val_acc',
    max_trials=100,
    overwrite=True)


In [20]:
callbacks=[keras.callbacks.EarlyStopping(monitor='val_acc', mode='max', patience=3)]
tuner.search(X_valid, y_valid, validation_data=(X_val, y_val), callbacks=callbacks, epochs=100)


Trial 10 Complete [00h 00m 02s]
val_acc: 0.6761904954910278

Best val_acc So Far: 0.8285714387893677
Total elapsed time: 00h 00m 33s
INFO:tensorflow:Oracle triggered exit


In [26]:
best_hp = tuner.get_best_hyperparameters()[0]
model = build_model(best_hp)
model.fit(X_valid, y_valid, epochs=1000)


Epoch 1/10
10/10 [==============================] - 1s 5ms/step - loss: 0.7031 - acc: 0.7067
Epoch 2/10
10/10 [==============================] - 0s 11ms/step - loss: 0.3770 - acc: 0.8833
Epoch 3/10
10/10 [==============================] - 0s 8ms/step - loss: 0.1700 - acc: 0.9433
Epoch 4/10
10/10 [==============================] - 0s 8ms/step - loss: 0.1085 - acc: 0.9600
Epoch 5/10
10/10 [==============================] - 0s 8ms/step - loss: 0.0464 - acc: 0.9867
Epoch 6/10
10/10 [==============================] - 0s 8ms/step - loss: 0.0736 - acc: 0.9700
Epoch 7/10
10/10 [==============================] - 0s 8ms/step - loss: 0.0569 - acc: 0.9833
Epoch 8/10
10/10 [==============================] - 0s 8ms/step - loss: 0.0562 - acc: 0.9833
Epoch 9/10
10/10 [==============================] - 0s 9ms/step - loss: 0.1154 - acc: 0.9733
Epoch 10/10
10/10 [==============================] - 0s 7ms/step - loss: 0.0620 - acc: 0.9833


In [27]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_25 (Dense)            (None, 384)               192384    
                                                                 
 batch_normalization_20 (Bat  (None, 384)              1536      
 chNormalization)                                                
                                                                 
 dense_26 (Dense)            (None, 288)               110880    
                                                                 
 batch_normalization_21 (Bat  (None, 288)              1152      
 chNormalization)                                                
                                                                 
 dense_27 (Dense)            (None, 320)               92480     
                                                                 
 batch_normalization_22 (Bat  (None, 320)             

In [30]:
# model.summary() をテキストに変換する
from io import StringIO

with StringIO() as buf:
    # StringIOに書き込む
    model.summary(print_fn=lambda x: buf.write(x + "\n"))
    # StringIOから取得
    model_summary_txt = buf.getvalue()
model_summary_txt

'Model: "sequential_5"\n_________________________________________________________________\n Layer (type)                Output Shape              Param #   \n=================================================================\n dense_25 (Dense)            (None, 384)               192384    \n                                                                 \n batch_normalization_20 (Bat  (None, 384)              1536      \n chNormalization)                                                \n                                                                 \n dense_26 (Dense)            (None, 288)               110880    \n                                                                 \n batch_normalization_21 (Bat  (None, 288)              1152      \n chNormalization)                                                \n                                                                 \n dense_27 (Dense)            (None, 320)               92480     \n                                      

In [31]:
path = f'./output/optuna/503_param-tune_nn.txt'    
s = model_summary_txt
with open(path, mode='w') as f:
    f.write(s)
